In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import csv
import seaborn as sns


In [ ]:
df = pd.read_csv("encoded_loan_data.csv", sep=',', engine='python', error_bad_lines=False, dtype='unicode')

Before visualizing this data we must ensure all processed data is numeric. 
<font color='red'>This will be more meaningful after enums</font> 

In [ ]:
(df["applicant_income_000s"]).describe()

In [ ]:
# applicant income histogram
df.income_bracket.hist()
plt.title('Applicant Income')
plt.xlabel('All applicant incomes')
plt.ylabel('Number of applicants')

In [ ]:
numeric_cols = df_test.columns

In [ ]:
assert [col in ['int64','float'] for col in [df[y].dtype for y in numeric_cols]]

In [ ]:
df.applicant_race_name_1.value_counts()

In [ ]:
# bucketing can't stay if we want to use plot function, need numeric data
df["income_bracket"] = df["income_bracket"].replace('(0, 18]', 1)
df["income_bracket"] = df["income_bracket"].replace('(18, 75]', 2)
df["income_bracket"] = df["income_bracket"].replace('(75, 153]', 3)
df["income_bracket"] = df["income_bracket"].replace('(153, 233]', 4)
df["income_bracket"] = df["income_bracket"].replace('(233, 416]', 5)
df["income_bracket"] = df["income_bracket"].replace('(416, 470]', 6)
df["income_bracket"] = df["income_bracket"].replace('(470, 9999]', 7)


In [ ]:
# need to re-cast column types right now because they are exported as csv after pre-processing
df["applicant_income_000s"]=df["applicant_income_000s"].astype(float)
df["action_taken_name"]=df["action_taken_name"].astype(float)
df["income_bracket"]=df["income_bracket"].astype(float)
df["loan_amount_000s"] = df["loan_amount_000s"].astype(float)

In [ ]:
df[df.applicant_race_name_1== "American Indian or Alaska Native"].groupby('income_bracket').action_taken_name.plot()
df[df.applicant_race_name_1=="Asian"].groupby('income_bracket').action_taken_name.plot()
df[df.applicant_race_name_1=="Black or African American"].groupby('income_bracket').action_taken_name.plot()
df[df.applicant_race_name_1=="Native Hawaiian or Other Pacific Islander"].groupby('income_bracket').action_taken_name.plot()
df[df.applicant_race_name_1=="Not applicable"].groupby('income_bracket').action_taken_name.plot()
df[df.applicant_race_name_1=="White"].groupby('income_bracket').action_taken_name.plot()
plt.title('Loan status by income, conditional on race')
plt.xlabel('Income')
plt.ylabel('Loan status')
plt.legend(['American Indian or Alaska Native', 'Asian', 'Black or African American'
           ,'Native Hawaiian or Other Pacific Islander', 'Not applicable','White'], loc='lower right')

